# Chapter 39: Build Adapters API

Next.js applications deploy to diverse platforms—from serverless edge networks to traditional containerized infrastructure. The Build Adapters API provides a mechanism to customize how your application is compiled, bundled, and prepared for specific deployment targets. Whether optimizing for Vercel's edge network, AWS Lambda's serverless constraints, or Docker containers, understanding build adapters ensures your application runs efficiently in any environment.

By the end of this chapter, you'll master configuring built-in output modes for different platforms, creating custom build adapters for specialized deployment targets, optimizing bundle sizes for serverless environments, handling platform-specific constraints (cold starts, memory limits), and implementing multi-target build pipelines.

## 39.1 Understanding Build Adapters

Build adapters determine how Next.js compiles your application into deployable artifacts. The default adapter targets Node.js, but modern deployments often require edge-compatible or static outputs.

### Built-in Output Configuration

```javascript
// next.config.js
/** @type {import('next').NextConfig} */
const nextConfig = {
  // Output modes determine the adapter behavior
  output: 'standalone', // 'standalone' | 'export' | 'server' (default)
  
  // Dist directory configuration
  distDir: 'dist',
  
  // Experimental build adapters
  experimental: {
    // Enable experimental compile mode
    compileMode: 'standard',
    
    // Server Actions configuration for edge
    serverActions: {
      bodySizeLimit: '2mb',
    },
    
    // App directory output optimization
    outputFileTracingRoot: process.cwd(),
    outputFileTracingExcludes: {
      '*': [
        'node_modules/@swc/core-linux-x64-gnu',
        'node_modules/@swc/core-linux-x64-musl',
        'node_modules/@esbuild/linux-x64',
      ],
    },
  },
  
  // Image optimization for static export
  images: {
    unoptimized: process.env.OUTPUT_MODE === 'export',
    remotePatterns: [
      { protocol: 'https', hostname: '**.amazonaws.com' },
    ],
  },
  
  // Tracing for minimal server bundles
  outputFileTracing: true,
};

module.exports = nextConfig;
```

### Environment-Based Output Switching

```typescript
// lib/build/config.ts
import type { NextConfig } from 'next';

export function getOutputConfig(): Partial<NextConfig> {
  const target = process.env.DEPLOYMENT_TARGET || 'vercel';
  
  const configs: Record<string, Partial<NextConfig>> = {
    vercel: {
      // Default Vercel configuration (no special output needed)
      output: undefined,
    },
    
    'aws-lambda': {
      output: 'standalone',
      experimental: {
        // Optimize for Lambda's 50MB limit
        serverMinification: true,
      },
    },
    
    docker: {
      output: 'standalone',
      distDir: '.next',
      // Disable telemetry in containers
      telemetry: false,
    },
    
    static: {
      output: 'export',
      distDir: 'dist',
      images: { unoptimized: true },
    },
    
    edge: {
      output: 'export', // Edge requires static or special handling
      experimental: {
        runtime: 'edge',
      },
    },
  };
  
  return configs[target] || configs.vercel;
}

// next.config.js
const { getOutputConfig } = require('./lib/build/config');

/** @type {import('next').NextConfig} */
const nextConfig = {
  ...getOutputConfig(),
  // Other config...
};

module.exports = nextConfig;
```

## 39.2 Output Modes Deep Dive

Configure Next.js for specific deployment architectures: Node.js servers, static sites, or standalone containers.

### Standalone Output for Containers

```dockerfile
# Dockerfile optimized for Next.js standalone output
FROM node:20-alpine AS base

# Install dependencies only when needed
FROM base AS deps
RUN apk add --no-cache libc6-compat
WORKDIR /app

COPY package.json package-lock.json* yarn.lock* pnpm-lock.yaml* ./
RUN \
  if [ -f yarn.lock ]; then yarn --frozen-lockfile; \
  elif [ -f package-lock.json ]; then npm ci; \
  elif [ -f pnpm-lock.yaml ]; then corepack enable pnpm && pnpm i --frozen-lockfile; \
  else echo "Lockfile not found." && exit 1; \
  fi

# Rebuild the source code only when needed
FROM base AS builder
WORKDIR /app
COPY --from=deps /app/node_modules ./node_modules
COPY . .

# Environment variables for build
ENV NEXT_TELEMETRY_DISABLED 1
ENV NODE_ENV production

RUN \
  if [ -f yarn.lock ]; then yarn build; \
  elif [ -f package-lock.json ]; then npm run build; \
  elif [ -f pnpm-lock.yaml ]; then corepack enable pnpm && pnpm run build; \
  else npm run build; \
  fi

# Production image, copy all the files and run next
FROM base AS runner
WORKDIR /app

ENV NODE_ENV production
ENV NEXT_TELEMETRY_DISABLED 1

RUN addgroup --system --gid 1001 nodejs
RUN adduser --system --uid 1001 nextjs

# Standalone output creates a minimal server.js in .next/standalone
COPY --from=builder /app/public ./public
COPY --from=builder --chown=nextjs:nodejs /app/.next/standalone ./
COPY --from=builder --chown=nextjs:nodejs /app/.next/static ./.next/static

USER nextjs

EXPOSE 3000

ENV PORT 3000
ENV HOSTNAME "0.0.0.0"

# Standalone server.js handles both static files and SSR
CMD ["node", "server.js"]
```

### Static Export with ISR Fallback

```javascript
// next.config.js - Static export with dynamic routes
const nextConfig = {
  output: 'export',
  
  // Generate dynamic routes at build time
  generateStaticParams: async () => {
    // This runs during build to create HTML files
    const posts = await fetchPosts();
    return posts.map((post) => ({
      slug: post.slug,
    }));
  },
  
  // Handle client-side data fetching for ISR-like behavior
  experimental: {
    // Enable SPA mode for static export (Next.js 14+)
    clientRouterFilter: true,
  },
  
  // Trailing slashes for static hosting compatibility
  trailingSlash: true,
  
  // Image optimization for static export
  images: {
    unoptimized: true, // Required for static export
    remotePatterns: [
      { hostname: 'cdn.example.com' },
    ],
  },
  
  // Headers for static hosting (converted to _headers for Netlify/Vercel)
  async headers() {
    return [
      {
        source: '/:path*',
        headers: [
          {
            key: 'X-Frame-Options',
            value: 'DENY',
          },
        ],
      },
    ];
  },
};

module.exports = nextConfig;
```

## 39.3 Custom Build Adapters

Create custom adapters for specialized deployment targets like AWS Lambda, Cloudflare Workers, or Deno Deploy.

### Custom Adapter Implementation

```typescript
// lib/build/custom-adapter.ts
import type { NextConfig } from 'next';

interface AdapterOptions {
  name: string;
  runtime: 'node' | 'edge' | 'deno';
  optimizeFor?: 'memory' | 'speed' | 'size';
}

export function createCustomAdapter(options: AdapterOptions): Partial<NextConfig> {
  const baseConfig: Partial<NextConfig> = {
    experimental: {
      // Enable custom runtime
      runtime: options.runtime,
      
      // Server Components external packages optimization
      serverComponentsExternalPackages: options.runtime === 'node' 
        ? ['sharp', 'prisma'] 
        : [],
    },
  };

  switch (options.runtime) {
    case 'edge':
      return {
        ...baseConfig,
        output: 'export', // Edge often requires static or limited dynamic
        experimental: {
          ...baseConfig.experimental,
          edgeRuntime: true,
          // Limit middleware bundle size
          allowMiddlewareResponseBody: true,
        },
      };
      
    case 'node':
      return {
        ...baseConfig,
        output: 'standalone',
        experimental: {
          ...baseConfig.experimental,
          // Node-specific optimizations
          serverMinification: options.optimizeFor === 'size',
          optimizePackageImports: ['lodash', 'date-fns'],
        },
      };
      
    default:
      return baseConfig;
  }
}

// Usage in next.config.js
const { createCustomAdapter } = require('./lib/build/custom-adapter');

module.exports = createCustomAdapter({
  name: 'aws-lambda',
  runtime: 'node',
  optimizeFor: 'size', // Critical for Lambda 50MB limit
});
```

### AWS Lambda Optimization

```javascript
// next.config.js for AWS Lambda (using OpenNext or similar)
const nextConfig = {
  output: 'standalone',
  
  // Critical for Lambda: Minimize bundle size
  experimental: {
    // Only include necessary files in trace
    outputFileTracingExcludes: {
      '*': [
        'node_modules/@swc/**',
        'node_modules/@next/swc/**',
        'node_modules/canvas/**',
        '.next/cache/**',
      ],
    },
    
    // Optimize memory usage
    serverMinification: true,
    
    // Disable features not supported in Lambda
    instrumentationHook: false,
  },
  
  // Image optimization (use AWS if possible)
  images: {
    loader: 'custom',
    loaderFile: './lib/image-loader.ts',
  },
  
  // Webpack optimization for Lambda cold starts
  webpack: (config, { isServer, nextRuntime }) => {
    if (isServer && nextRuntime === 'nodejs') {
      // Exclude native binaries that bloat the bundle
      config.externals.push(
        'sharp',
        'node-gyp',
        'canvas'
      );
      
      // Bundle specific modules for Lambda compatibility
      config.resolve.fallback = {
        ...config.resolve.fallback,
        fs: false,
        net: false,
        tls: false,
        'aws-sdk': false, // Use Lambda's built-in SDK
      };
    }
    
    return config;
  },
};

module.exports = nextConfig;
```

```typescript
// lib/image-loader.ts
// Custom image loader for AWS S3/CloudFront
interface ImageLoaderProps {
  src: string;
  width: number;
  quality?: number;
}

export default function awsImageLoader({ src, width, quality }: ImageLoaderProps) {
  const url = new URL(src, process.env.NEXT_PUBLIC_CDN_URL);
  
  // Add AWS Image Processing parameters
  url.searchParams.set('w', width.toString());
  url.searchParams.set('q', (quality || 75).toString());
  url.searchParams.set('f', 'auto'); // Auto format (WebP/AVIF)
  
  return url.href;
}
```

## 39.4 Multi-Target Build Pipeline

Configure builds for multiple deployment targets simultaneously.

```typescript
// scripts/build-all.ts
import { execSync } from 'child_process';
import fs from 'fs/promises';
import path from 'path';

const targets = [
  { name: 'vercel', env: { OUTPUT_MODE: 'vercel' } },
  { name: 'docker', env: { OUTPUT_MODE: 'docker' } },
  { name: 'static', env: { OUTPUT_MODE: 'static' } },
];

async function buildForTarget(target: typeof targets[0]) {
  console.log(`\n🔨 Building for ${target.name}...\n`);
  
  const env = { ...process.env, ...target.env };
  
  try {
    // Clean previous build
    await fs.rm('.next', { recursive: true, force: true });
    
    // Build with target-specific config
    execSync('next build', {
      stdio: 'inherit',
      env,
    });
    
    // Copy to target-specific output directory
    const outputDir = `dist/${target.name}`;
    await fs.mkdir(outputDir, { recursive: true });
    await fs.cp('.next', outputDir, { recursive: true });
    
    // Target-specific post-processing
    if (target.name === 'docker') {
      await fs.cp('Dockerfile', `${outputDir}/Dockerfile`);
    }
    
    console.log(`✅ Build complete for ${target.name}`);
  } catch (error) {
    console.error(`❌ Build failed for ${target.name}:`, error);
    process.exit(1);
  }
}

async function main() {
  for (const target of targets) {
    await buildForTarget(target);
  }
  
  console.log('\n🎉 All builds completed successfully!');
}

main();
```

## 39.5 Advanced Build Configuration

Fine-tune builds for production performance and debugging.

### Build-Time Feature Flags

```typescript
// lib/features.ts
export const features = {
  // Build-time feature flags
  enableAnalytics: process.env.ENABLE_ANALYTICS === 'true',
  enableExperimentalCache: process.env.EXPERIMENTAL_CACHE === 'true',
  runtime: process.env.NEXT_RUNTIME || 'nodejs',
  
  // Adapter detection
  isEdgeRuntime: process.env.NEXT_RUNTIME === 'edge',
  isStaticExport: process.env.OUTPUT_MODE === 'static',
} as const;

// Usage in components
import { features } from '@/lib/features';

export function AnalyticsWrapper({ children }: { children: React.ReactNode }) {
  // Tree-shaken in builds where feature is disabled
  if (!features.enableAnalytics) {
    return <>{children}</>;
  }
  
  return (
    <AnalyticsProvider>
      {children}
    </AnalyticsProvider>
  );
}
```

### Build Analysis and Optimization

```javascript
// next.config.js - Bundle analysis
const withBundleAnalyzer = require('@next/bundle-analyzer')({
  enabled: process.env.ANALYZE === 'true',
});

const nextConfig = {
  // Experimental build optimizations
  experimental: {
    // Optimize package imports (automatic tree-shaking)
    optimizePackageImports: [
      'lodash',
      'date-fns',
      '@mui/material',
      '@mui/icons-material',
    ],
    
    // Turbopack-specific optimizations
    turbo: {
      rules: {
        '*.svg': {
          loaders: ['@svgr/webpack'],
          as: '*.js',
        },
      },
      resolveAlias: {
        'lodash': 'lodash-es',
      },
    },
    
    // Memory optimization for large builds
    maxMemory: 8192, // MB
  },
  
  // Compiler optimizations
  compiler: {
    // Remove console logs in production
    removeConsole: process.env.NODE_ENV === 'production' ? {
      exclude: ['error', 'warn'],
    } : false,
    
    // React optimizations
    reactRemoveProperties: process.env.NODE_ENV === 'production',
    reactStrictMode: true,
  },
  
  // Modularize imports for large libraries
  modularizeImports: {
    'lodash': {
      transform: 'lodash/{{member}}',
    },
    '@mui/material': {
      transform: '@mui/material/{{member}}',
    },
    '@mui/icons-material': {
      transform: '@mui/icons-material/{{member}}',
    },
  },
};

module.exports = withBundleAnalyzer(nextConfig);
```

## Key Takeaways from Chapter 39

1. **Output Modes**: Use `output: 'standalone'` for Docker/container deployments (creates minimal `server.js`), `output: 'export'` for static hosting (generates HTML files), and default `output: 'server'` for serverless platforms like Vercel that handle the runtime automatically.

2. **Bundle Optimization**: Configure `experimental.outputFileTracingExcludes` to remove unnecessary native binaries (SWC, esbuild) from serverless bundles, critical for AWS Lambda's 50MB limit. Use `modularizeImports` to tree-shake large libraries like MUI or Lodash automatically.

3. **Custom Adapters**: Create adapter factory functions that return NextConfig objects based on `DEPLOYMENT_TARGET` environment variables. Handle platform-specific constraints like Lambda memory limits (128MB-10GB) or Edge runtime restrictions (no Node.js built-ins).

4. **Static Export Limitations**: When using `output: 'export'`, remember that API Routes, Middleware, and Server Actions requiring dynamic execution won't work. Use client-side data fetching with `useEffect` and static revalidation via `generateStaticParams` for dynamic content.

5. **Docker Optimization**: Multi-stage builds with Alpine Linux reduce image size by 80%. The standalone output includes only necessary `node_modules` files traced by Next.js, eliminating the need to copy entire `node_modules` directories.

6. **Image Loaders**: Custom loaders (`images.loader: 'custom'`) are required for static exports or non-Vercel hosts. Implement platform-specific URL transformations (CloudFront, Cloudflare Images, Imgix) to leverage CDN optimization features.

7. **Multi-Target Builds**: Script sequential builds with environment variable switching to generate artifacts for multiple platforms (Vercel, Docker, AWS) from the same source. Clean `.next` directory between builds to prevent cache pollution across targets.

## Coming Up Next

**Chapter 40: Next.js DevTools**

With your build pipeline configured for multiple deployment targets, it's time to explore the developer experience improvements available in Next.js. In Chapter 40, we'll examine the Next.js DevTools integration, React DevTools profiling for Server Components, performance debugging techniques, the new Next.js MCP (Model Context Protocol) server for AI assistance, and network monitoring tools. You'll learn how to leverage these tools for faster debugging and deeper application insights.